In [ ]:
#  Uhlig's Exampl6.m
#  A Small Open Economy Stochastic Neoclassical 
#  Growth Model with Adjustment Costs for Capital

import numpy as np

Z_bar = 1
NPV_frac = 0.5
rho = 0.36
delta = 0.025
R_bar = 1.01
eta = 1.0
theta = 0.8
psi_z = 0.95
sigma_z = 0.712
psi_r = 0.95
sigma_r = 1.0
corr_z_r = 0

betta = 1.0/R_bar
XK_bar = ((1+delta)**theta - 1)**(1.0/theta)
FK_omt = (1+delta)**(1-theta)
FX_omt = FK_omt/XK_bar**(1-theta)
YK_bar = (R_bar - FK_omt + delta)/(rho*FX_omt)
K_bar = (Z_bar/YK_bar)**(1.0/(1-rho))
Y_bar = Z_bar*K_bar**rho
X_bar = XK_bar*K_bar
F_bar = (1+delta)*K_bar
A_bar = NPV_frac * (Y_bar/(R_bar - 1))
C_bar = Y_bar - X_bar + (R_bar - 1)*A_bar

VARNAMES = ['capital ',
                         'assets ',
                         'consumption ',
                         'investment ',
                         'output ',
                         'intermediate capital ',
                         'technology ',
                         'return ' ]

AA = np.vstack(([ 0, -A_bar],
                           [0, 0], 
                           [-1, 0],    
                           [0, 0]))

BB = np.vstack(([0, R_bar*A_bar],
                           [rho, 0],
                           [-delta, 0],       
                           [K_bar**theta, 0]))

CC = np.vstack(([ -C_bar, -X_bar, Y_bar, 0],
                           [0, 0, -1, 0],
                           [0, 0, 0,(1+delta)],
                           [0, X_bar**theta, 0, (-F_bar**theta) ]))


DD = np.vstack(([0,R_bar*A_bar],
                           [1,0],
                           [0,0],          
                           [0,0]))

FF = np.vstack(([0,0], 
                          [0,0]))

GG = np.vstack(([0, 0],
                           [(-((rho/R_bar)*YK_bar*FX_omt + (1-theta)*FK_omt/R_bar) ),0]))

HH = np.vstack(([0,0],
                           [0,0]))

JJ = np.vstack(([-eta, 0, 0, 0],     
                          [-eta,(1-theta)*(FK_omt-delta)/R_bar,(rho/R_bar)*YK_bar*FX_omt,delta*(1-theta)/R_bar]))

KK = np.vstack(([eta, 0, 0, 0],
                           [eta,theta-1,0,1-theta]))

LL = np.vstack(([0,1],
                          [0,0]))

MM = np.vstack(([0,0],
                            [0,0]))

NN = np.vstack(([psi_z, 0],
                           [0, psi_r ]))

Sigma = np.vstack(([sigma_z**2, corr_z_r*sigma_z*sigma_r], 
                                [corr_z_r*sigma_z*sigma_r, sigma_r**2]))

Sigma_EPS = Sigma


np.set_printoptions(precision=4, suppress=True)

%run Uhlig_solver_abreviated.ipynb



In [ ]:
#   This is filename:  Uhlig_impulse_resp_all.ipynb
#   It computes for impulse responses to 1% 
#   innovations for all exogenous stochastic 
#   processes in the model.

#  The inputs into this notebook are the model
#  and its solution.  See  uhlig_solver_abreviated.ipynb

#  The output of this notebook are the mactices
#  "Resp_mat"   and   "Resp_mat_named"
#  which contain impulse response values 



import numpy as np
from numpy import hstack, vstack, zeros, dot, eye, kron

import scipy as sp
from scipy import linalg as la
from scipy.linalg import null_space
import scipy.optimize as opt

import matplotlib.pyplot as plt
import pylab

np.set_printoptions(precision=4, suppress=True)


##  Choose number of periods over which 
##  impulse responses are to be computed 

Horizon = 12
#print('Horizon = ', Horizon)


m_states = QQ.shape[0]
k_exog = QQ.shape[1]
n_endog = SS.shape[0]
#print('m_states = ', m_states)
#print('n_endog = ', n_endog)
#print('k_exog = ', k_exog)



Period_list = np.matrix(range(0,Horizon+1)).reshape(1, Horizon+1)

shock_counter = 0
time_counter = 0


Resp_mat = Period_list

for shock_counter in range(k_exog):

    Response =  np.zeros((m_states+n_endog+k_exog)*Horizon).reshape(m_states+n_endog+k_exog, Horizon)
    Response = np.array(Response)
    Response[m_states+n_endog+shock_counter,0] = 1
    #print('shock_counter = ', shock_counter)
    #print('Response = ', Response)
                    
    II_Lag_01 = np.hstack(((PP, np.zeros(m_states*n_endog).reshape(m_states, n_endog), np.zeros(m_states*k_exog).reshape(m_states, k_exog))))
    II_Lag_02 = np.hstack(((RR, np.zeros(n_endog*n_endog).reshape(n_endog, n_endog), np.zeros(n_endog*k_exog).reshape(n_endog, k_exog))))
    II_Lag_03 = np.hstack(((np.zeros(k_exog*(m_states+n_endog)).reshape(k_exog, (m_states+n_endog)), NN)))
    II_Lag = np.vstack((II_Lag_01, II_Lag_02, II_Lag_03))
    #print('II_Lag = ', II_Lag)
                    
    II_Contemp_01 = np.eye(m_states+n_endog+k_exog)
    II_Contemp_02 = np.hstack((np.zeros(m_states*(m_states+n_endog)).reshape(m_states, (m_states+n_endog)), QQ))
    II_Contemp_03 = np.hstack((np.zeros(n_endog*(m_states+n_endog)).reshape(n_endog, (m_states+n_endog)), SS))
    II_Contemp_04 = np.hstack((np.zeros(k_exog*(m_states+n_endog)).reshape(k_exog, (m_states+n_endog)), np.zeros(k_exog*k_exog).reshape(k_exog, k_exog)))
    II_Contemp = II_Contemp_01+ np.vstack((II_Contemp_02, II_Contemp_03, II_Contemp_04))
    #print('II_Contemp = ', II_Contemp)
                    
    Response_0 = np.array(Response[:,0]).reshape((m_states+n_endog+k_exog),1)
    #print('Response_0 = ', Response_0)
    #print('Response_0 shape = ', Response_0.shape)
    
    Response_1 = dot(II_Contemp, Response_0).reshape((m_states+n_endog+k_exog),1)
    #print('Response_1 = ', Response_1)
    #print('Response_1 shape = ', Response_1.shape)
    
    Response = Response_1
                    
    for time_counter in range(0, Horizon):
        # print('time_counter = ', time_counter)
        Resp_col = dot(dot(II_Contemp, II_Lag), Response[:,time_counter -1])
        Response = np.hstack((Response, Resp_col))
        # print('time_counter = ', time_counter)
        # print(Response)
        
    Resp_mat = np.vstack((Resp_mat, Response, Period_list))

print('   ')
print('Resp_mat = ',Resp_mat)

    

VN_temp= np.matrix(VARNAMES)
VARNAMES = VN_temp.reshape((m_states+k_exog+n_endog),1)
VAR_NAMES = np.vstack((['Period'],VARNAMES))

for shock_counter in range(1,k_exog):    
    VAR_NAMES = np.vstack((VAR_NAMES,['Period'],VARNAMES))

VAR_NAMES =  np.vstack((VAR_NAMES,['Period']))

Resp_mat_named = np.hstack((VAR_NAMES, Resp_mat))     
print('   ')
print('Resp_mat_named = ', Resp_mat_named)



print('  ')    
print('Done. You have the matrices "Resp_mat" and "Resp_mat_named" ')
print('which contains impulse reponses to 1% innovations')
print('for all exogenous stochastic processes in the model.')
print('If you want, you can download these into an excel file')
print('or use them to make plots.')
print('  ')